In [2]:
import tensorflow as tf
import importlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
from datetime import datetime
import utils_siamese1
import model1
import results_generator
importlib.reload(utils_siamese1)
importlib.reload(model1)
importlib.reload(results_generator)
from utils_siamese1 import *
from model1 import *
from results_generator import *
tf.test.is_built_with_cuda()

True

In [2]:
# This code is only needed if the dataset is stored in google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Check if gpu exists
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [3]:
# This code is only needed if the dataset is stored in google drive and it is 7z compressed
!p7zip -d /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/Thesis/                                         1 file, 263683120 bytes (252 MiB)

Extracting archive: /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
--
Path = /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
Type = 7z
Physical Size = 263683120
Headers Size = 344455
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% 952 - Sketch-Icon-Dataset/icon/mathematics/wave-square.jpg                                                                 2% 1457 - Sketch-Icon-Dataset/sketch/accessibility/low-visio

In [3]:
icon_name_category, sketch_name_category = get_icons_and_sketches()
icon_dictionary, sketch_dictionary = load_icons_sketches_dic(icon_name_category, sketch_name_category)

In [4]:
def shuffle_in_unison(a, b):
    n_elem = a.shape[0]
    indeces = np.random.permutation(n_elem)
    return a[indeces], b[indeces]

# Split the dataset into train and test with randomness
# or load the train and test dataset which is created as numpy arrays.
# The purpose of the predefined datasets is to have the same datasets for training
# the classification of sketches and the sketch-icon retrieval, so the test set will be the same,
# and unknown for both trainings
random_generation = False
if random_generation:
    positive_pairs, positive_labels = positive_pairs_generator(icon_name_category, sketch_name_category)
    positive_pairs = shuffle(positive_pairs)

    len_data = len(positive_pairs)
    p_train=0.9
    p_test=0.1
    num_train = int(np.ceil(len_data*p_train))
    num_test = int(np.floor(len_data*p_test))

    positive_pairs_Train, positive_labels_Train = positive_pairs[:num_train], positive_labels[:num_train]
    possible_negative_pairs_Train = possible_negative_pairs_generator(positive_pairs_Train, icon_name_category)
    negative_pairs_Train, negative_labels_Train = negative_pairs_generator(positive_pairs_Train, possible_negative_pairs_Train)
    pairs_Train = np.concatenate((positive_pairs_Train, negative_pairs_Train), axis=0)
    labels_Train = np.concatenate((positive_labels_Train, negative_labels_Train), axis=0)
    pairs_Train, labels_Train = shuffle_in_unison(pairs_Train, labels_Train)

    positive_pairs_Test = positive_pairs[-num_test:]
    sketches_Test = positive_pairs_Test[:, [0, 1]]

    icons_Test = positive_pairs_Test[:, [2, 3]]
    # create the icons test dataset by removing the dublicates of the array
    _, unique_indices = np.unique(icons_Test[:,0], return_index=True)
    unique_icons_Test = icons_Test[unique_indices]

    print(f'We have {len(pairs_Train)} samples in the training set.')
    print(f'We have {len(sketches_Test)} sketches in the test set.')
    print(f'We have {len(unique_icons_Test)} unique icons in the test set.')
else:
    sketch_name_category_Train = load_train_set()
    sketch_name_category_Test = load_test_set()

    positive_pairs_Train, positive_labels_Train =  positive_pairs_generator(icon_name_category, sketch_name_category_Train)
    possible_negative_pairs_Train = possible_negative_pairs_generator(positive_pairs_Train, icon_name_category)
    negative_pairs_Train, negative_labels_Train = negative_pairs_generator(positive_pairs_Train, possible_negative_pairs_Train)
    pairs_Train = np.concatenate((positive_pairs_Train, negative_pairs_Train), axis=0)
    labels_Train = np.concatenate((positive_labels_Train, negative_labels_Train), axis=0)
    pairs_Train, labels_Train = shuffle_in_unison(pairs_Train, labels_Train)

    positive_pairs_Test,_ =  positive_pairs_generator(icon_name_category, sketch_name_category_Test)

    sketches_Test = positive_pairs_Test[:, [0, 1]]

    icons_Test = positive_pairs_Test[:, [2, 3]]
    
    # create the icons test dataset by removing the dublicates of the array
    _, unique_indices = np.unique(icons_Test[:,0], return_index=True)
    unique_icons_Test = icons_Test[unique_indices]

    print(f'We have {len(pairs_Train)} samples in the training set.')
    print(f'We have {len(sketches_Test)} sketches in the test set.')
    print(f'We have {len(unique_icons_Test)} unique icons in the test set.')

We have 57252 samples in the training set.
We have 3180 sketches in the test set.
We have 1178 unique icons in the test set.


In [14]:
choices = ["Siamese Loss", "Siamese Loss-CWI"] # CWI = Classification Weights Initialization
model_name = "MyNet"
create_directory(model_name + "/")
now = datetime.now()
date_time_folder = now.strftime("%d-%m-%Y %H-%M-%S")
train_weights_folder = "Train Weights"
choice = 1

# choice 0 is the siamese loss without initialization of classification weights - one model (mynet) with shared weights
# choice 1 is the siamese loss without initialization of classification weights - two models (mynet) initialized classification weights and update with the same grads
if choice == 0:
  # Creation of folders for storing stats, weights and the hyperparameters of the model
  create_directory(model_name + "/" + choices[choice])
  current_run_path = model_name + "/" + choices[choice] + "/" + date_time_folder + "/"
  create_directory(current_run_path)
  train_weights_path = model_name + "/" + choices[choice] + "/" + date_time_folder + "/" + train_weights_folder
  create_directory(train_weights_path)
  #=====================================================================================

  # Hyperparameter initializations
  BATCH_SIZE = 256
  num_epochs = 1000
  margin = 1
  learning_rate = 0.0001

  # Store the hyperparameters of the model into a file to remember what I used in a specific run
  write_hyperparameters_in_file(current_run_path, learning_rate, BATCH_SIZE, margin)
  
  # Initializations of optimizer, loss and network and set the backend (model variables) to float32
  optimizer = tf.keras.optimizers.Adam(learning_rate)
  tf.keras.backend.set_floatx('float32')
  siameseModel = mynet()
  loss = siamese_loss

  # Function which is responsible to calculate the loss and update the grads
  def train_step(sketches, icons, labels):
      with tf.GradientTape() as tape:
          sketch_features,_ = siameseModel(sketches, training = True)
          icon_features,_ = siameseModel(icons, training = True)  
          tape.watch(sketch_features)
          tape.watch(icon_features)
          current_loss = loss(sketch_features, icon_features, labels)
      grads = tape.gradient(current_loss, siameseModel.trainable_variables)
      optimizer.apply_gradients((grad, var) for (grad, var) in zip(grads, siameseModel.trainable_variables) if grad is not None)
      return current_loss

elif choice == 1:
  # Creation of folders for storing stats, weights and the hyperparameters of the model
  create_directory(model_name + "/" + choices[choice])
  current_run_path = model_name + "/" + choices[choice] + "/" + date_time_folder + "/"
  create_directory(current_run_path)
  train_weights_path = model_name + "/" + choices[choice] + "/" + date_time_folder + "/" + train_weights_folder
  create_directory(train_weights_path)
  #=====================================================================================

  # Hyperparameter initializations
  BATCH_SIZE = 256
  num_epochs = 1000
  margin = 1
  learning_rate = 0.0001

  # Store the hyperparameters of the model into a file to remember what I used in a specific run
  write_hyperparameters_in_file(current_run_path, learning_rate, BATCH_SIZE, margin)

  # Initializations of optimizer, loss and network and set the backend (model variables) to float32
  optimizer = tf.keras.optimizers.Adam(learning_rate)
  tf.keras.backend.set_floatx('float32')
  weights_path_sketch = "/content/gdrive/MyDrive/Thesis/MyNet/Sketch Classification/24-03-2021 17-39-23/Train Weights/"
  weights_path_icon = "/content/gdrive/MyDrive/Thesis/MyNet/Icon Classification/24-03-2021 17-34-23/Train Weights/"
  iconClassificationModel = mynet(NUM_CLASSES=66)
  iconClassificationModel.load_weights(weights_path_icon + 'iconClassification')
  sketchClassificationModel = mynet(NUM_CLASSES=66)
  sketchClassificationModel.load_weights(weights_path_sketch + 'sketchClassification')
  loss = contrastive_loss

  # Function which is responsible to calculate the loss and update the grads
  def train_step(sketches, icons, labels, margin):
      with tf.GradientTape(persistent=True) as tape:
          sketch_features,_ = sketchClassificationModel(sketches, training = True)
          icon_features,_ = iconClassificationModel(icons, training = True)
          labels = tf.convert_to_tensor(labels, dtype=tf.float32)
          tape.watch(sketch_features)
          tape.watch(icon_features)
          tape.watch(labels)
          current_loss = loss(sketch_features, icon_features, labels, margin)
      grads = tape.gradient(current_loss, sketchClassificationModel.trainable_variables)
      optimizer.apply_gradients((grad, var) for (grad, var) in zip(grads, sketchClassificationModel.trainable_variables) if grad is not None)
      grads = tape.gradient(current_loss, iconClassificationModel.trainable_variables)
      optimizer.apply_gradients((grad, var) for (grad, var) in zip(grads, iconClassificationModel.trainable_variables) if grad is not None)
      return current_loss

Creation of the directory  failedMyNet/
Creation of the directory  failedMyNet/Siamese Loss-CWI
Successfully created the directory  MyNet/Siamese Loss-CWI/05-04-2021 12-06-06/
Successfully created the directory  MyNet/Siamese Loss-CWI/05-04-2021 12-06-06/Train Weights


In [15]:
top_acc = 0
top_acc1 = 0
top_acc10 = 0
for epoch in range(num_epochs):

    # Training loop throught the training dataset
    epoch_loss_avg = tf.keras.metrics.Mean()
    for i in range(0, len(pairs_Train), BATCH_SIZE):
        batch_pairs = pairs_Train[i:i+BATCH_SIZE]
        batch_labels = labels_Train[i:i+BATCH_SIZE]
        sketches, icons = get_batch(batch_pairs, icon_dictionary, sketch_dictionary)
        loss_value = train_step(sketches, icons, batch_labels, margin)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))

    if epoch%1==0:
      # Test loop through the test datatet
        acc_1 = 0
        acc_10 = 0

        # extract features for sketches
        sketch_representations = []
        for j in range(0, len(sketches_Test), BATCH_SIZE):
            batch_sketches = sketches_Test[j:j+BATCH_SIZE]
            sketches_array = get_batch_sketches(batch_sketches)
            if choice == 0:
              sketch_repr,_ =  siameseModel(sketches_array, training = False)
            elif choice == 1:
              sketch_repr,_ =  sketchClassificationModel(sketches_array, training = False)
            sketch_representations.append(sketch_repr)
        sketch_representations = np.vstack(sketch_representations)

        # extract features for icons
        icon_representations = []
        for j in range(0, len(unique_icons_Test), BATCH_SIZE):
            batch_icons = unique_icons_Test[j:j+BATCH_SIZE]
            icons_array = get_batch_icons(batch_icons)
            if choice == 0:
              icons_repr,_ =  siameseModel(icons_array, training = False)
            elif choice == 1:
              icons_repr,_ =  iconClassificationModel(icons_array, training = False)
            icon_representations.append(icons_repr)
        icon_representations = np.vstack(icon_representations)

        # check using euclidean distance the top 1 and top 10 accuracy
        for k in range(len(sketch_representations)):
            sketch_repr = sketch_representations[k]
            sketch_representations_tile = np.tile(sketch_repr, len(unique_icons_Test)).reshape(len(unique_icons_Test), 64)
            diff = np.sqrt(np.mean((sketch_representations_tile - icon_representations)**2, -1))
            top_k = np.argsort(diff)[:10]
            
            for j in range(len(top_k)):
                index = top_k[j]
                if j == 0 and sketches_Test[k][0].split("_")[0] == unique_icons_Test[index][0].replace(".jpg",""):
                    acc_1 = acc_1 + 1
                    acc_10 = acc_10 + 1
                    break
                elif sketches_Test[k][0].split("_")[0] == unique_icons_Test[index][0].replace(".jpg",""):
                    acc_10 = acc_10 + 1
                    break
        if top_acc < acc_1 + acc_10:
          if choice == 0:
            save_weights(tripleModel, train_weights_path + "/SiameseWithCWI")
          elif choice == 1:
            save_weights(iconClassificationModel, train_weights_path + "/iconSiameseWithCWI")
            save_weights(sketchClassificationModel, train_weights_path + "/sketchSiamseseWithCWI")
          top_acc = acc_1 + acc_10
          top_acc1 = acc_1
          top_acc10 = acc_10
        print("Accuracy of top 1: " + str(acc_1/len(sketches_Test)))
        print("Accuracy of top 10: " + str(acc_10/len(sketches_Test)))
        write_triplet_stats_in_file(current_run_path, epoch, epoch_loss_avg.result(), acc_1/len(sketches_Test), acc_10/len(sketches_Test))
    
    # generate different negative pairs and shuffle
    negative_pairs_Train, negative_labels_Train = negative_pairs_generator(positive_pairs_Train, possible_negative_pairs_Train)
    pairs_Train = np.concatenate((positive_pairs_Train, negative_pairs_Train), axis=0)
    labels_Train = np.concatenate((positive_labels_Train, negative_labels_Train), axis=0)
    pairs_Train, labels_Train = shuffle_in_unison(pairs_Train, labels_Train)
  
print("The best accuracy of top 1: " + str(top_acc1/len(sketches_Test)))
print("The best accuracy of top 10: " + str(top_acc10/len(sketches_Test)))

Epoch 0: Loss: 440.417
Accuracy of top 1: 0.0018867924528301887
Accuracy of top 10: 0.010377358490566037
Epoch 1: Loss: 22.568
Accuracy of top 1: 0.0018867924528301887
Accuracy of top 10: 0.009748427672955974
Epoch 2: Loss: 5.752
Accuracy of top 1: 0.0025157232704402514
Accuracy of top 10: 0.015408805031446541
Epoch 3: Loss: 2.486
Accuracy of top 1: 0.002830188679245283
Accuracy of top 10: 0.016037735849056604
Epoch 4: Loss: 1.332
Accuracy of top 1: 0.00220125786163522
Accuracy of top 10: 0.016352201257861635
Epoch 5: Loss: 0.829
Accuracy of top 1: 0.00220125786163522
Accuracy of top 10: 0.016037735849056604
Epoch 6: Loss: 0.562
Accuracy of top 1: 0.0012578616352201257
Accuracy of top 10: 0.01289308176100629
Epoch 7: Loss: 0.418
Accuracy of top 1: 0.0009433962264150943
Accuracy of top 10: 0.011320754716981131
Epoch 8: Loss: 0.324
Accuracy of top 1: 0.0012578616352201257
Accuracy of top 10: 0.010377358490566037
Epoch 9: Loss: 0.266
Accuracy of top 1: 0.0006289308176100629
Accuracy of to

KeyboardInterrupt: ignored

In [ ]:
if epoch%1==0:
    acc_1 = 0
    acc_10 = 0
    sketch_representations = []
    for j in range(0, len(sketches_Test), BATCH_SIZE):
        batch_sketches = sketches_Test[j:j+BATCH_SIZE]
        sketches_array = get_batch_sketches(batch_sketches)
        sketch_repr,_ =  sketchClassificationModel(sketches_array, training = False)
        sketch_representations.append(sketch_repr)
    sketch_representations = np.vstack(sketch_representations)
    
    icon_representations = []
    for j in range(0, len(unique_icons_Test), BATCH_SIZE):
        batch_icons = unique_icons_Test[j:j+BATCH_SIZE]
        icons_array = get_batch_icons(batch_icons)
        icons_repr,_ =  iconClassificationModel(icons_array, training = False)
        icon_representations.append(icons_repr)

    icon_representations = np.vstack(icon_representations)

    for k in range(len(sketch_representations)):
        sketch_repr = sketch_representations[k]
        sketch_representations_tile = np.tile(sketch_repr, len(unique_icons_Test)).reshape(len(unique_icons_Test), 64)
        diff = np.sqrt(np.mean((sketch_representations_tile - icon_representations)**2, -1))
        top_k = np.argsort(diff)[:10]
        # img = sketch_dictionary[sketches_Test[k][0]]
        # plt.imshow(img)
        # plt.xticks([])
        # plt.yticks([])
        # plt.show()
        
        
        for j in range(len(top_k)):
            index = top_k[j]
            if j == 0 and sketches_Test[k][0].split("_")[0] == unique_icons_Test[index][0].replace(".jpg",""):
                acc_1 = acc_1 + 1
                acc_10 = acc_10 + 1
                #break
            elif sketches_Test[k][0].split("_")[0] == unique_icons_Test[index][0].replace(".jpg",""):
                acc_10 = acc_10 + 1
                #break
        #     img = icon_dictionary[unique_icons_Test[index][0]]
        #     plt.subplot(1, 10, j+1)
        #     plt.imshow(img)
        #     plt.xticks([])
        #     plt.yticks([])
        # plt.show()     
    print("Accuracy of top 1: " + str(acc_1/len(sketches_Test)))
    print("Accuracy of top 10: " + str(acc_10/len(sketches_Test)))

            

Accuracy of top 1: 0.005031446540880503
Accuracy of top 10: 0.03238993710691824
